# Step 1: Load data

In [2]:
import pandas as pd
import numpy as np

In [3]:
rec1data = pd.read_csv("../data/processed/rec1data.csv")
psychdata = pd.read_csv("../data/processed/psychdata.csv")

# Step 2: Clean data

In [4]:
# Incorrect data
psychdata["date_contact"][3597] = None
psychdata["date_term"][3597] = None

<ipython-input-4-4450631df2b6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  psychdata["date_contact"][3597] = None
<ipython-input-4-4450631df2b6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  psychdata["date_term"][3597] = None


In [5]:
# Convert dates to usable format
psychdata["date_contact"] = psychdata["date_contact"] + 19000000
psychdata["date_term"] = psychdata["date_term"] + 19000000

In [6]:
# Remove unneeded columns
psychdata = psychdata[["id", "contact", "date_contact", "diagnosis", "service", "date_term", "reason_term"]]
del rec1data["rid"]

In [7]:
# Remove unneeded rows from matched non-prisoner cases
rec1data = rec1data[rec1data["crime01"] > 0]

In [9]:
# Merge with psychdata, which will delete rows from non-prisoner cases
dta = rec1data.merge(psychdata, on = "id", how = "left")
dta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3479 entries, 0 to 3478
Data columns (total 31 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    3479 non-null   int64  
 1   id            3479 non-null   int64  
 2   sex           3479 non-null   int64  
 3   birth         3479 non-null   int64  
 4   race          3479 non-null   int64  
 5   educ          3479 non-null   int64  
 6   marital       3479 non-null   int64  
 7   children      3479 non-null   int64  
 8   occupat       3479 non-null   int64  
 9   arrests       3479 non-null   int64  
 10  jail          3479 non-null   int64  
 11  crime01       3479 non-null   int64  
 12  crime02       3479 non-null   int64  
 13  crime03       3479 non-null   int64  
 14  crime04       3479 non-null   int64  
 15  crime05       3479 non-null   int64  
 16  crime06       3479 non-null   int64  
 17  crime07       3479 non-null   int64  
 18  crime08       3479 non-null 

# Step 3: Process data

In [11]:
# Create length of treatment variable
dta["date_contact"] = pd.to_datetime(dta["date_contact"], format='%Y%m%d', errors = "coerce")
dta["date_term"] = pd.to_datetime(dta["date_term"], format='%Y%m%d', errors = "coerce")
dta["length_treat"] = (dta["date_term"] - dta["date_contact"]).dt.days
dta["length_treat"].describe()

count    3155.000000
mean       37.560697
std       118.498069
min         0.000000
25%         0.000000
50%         5.000000
75%        16.000000
max      2590.000000
Name: length_treat, dtype: float64

In [10]:
# Create number of crimes variable
dta['crime_num'] = (dta.iloc[:,11:24] != 0).sum(axis=1)
dta["crime_num"].describe()

count    3479.000000
mean        1.791894
std         2.000096
min         1.000000
25%         1.000000
50%         1.000000
75%         2.000000
max        13.000000
Name: crime_num, dtype: float64

# Step 4: Wrangle data

I had to do this step earlier to remove the rows that I'm not using.